# Experimenting with MongoDB Docker container

## Set up of Docker container on local machine

For this experiment I created a local folder with the following structure on my machine. <br>

```
docker_test
    └── data
        ├── csv_data
        │   ├── 1652951631.csv
        │   ├── 1652951755.csv
        │   ├── 1652951877.csv
        │   ├── 1652951992.csv
        │   ├── 1652952112.csv
        │   ├── 1652952240.csv
        │   ├── 1652952353.csv
        │   ├── 1652952472.csv
        │   ├── 1652952598.csv
        │   ├── 1652952711.csv
        │   ├── 1652952837.csv
        │   ├── 1652952961.csv
        │   ├── 1652953072.csv
        │   ├── 1652953199.csv
        │   ├── 1652953311.csv
        │   ├── 1652953435.csv
        │   ├── 1652953552.csv
        │   └── 1652953679.csv
        └── mongo_db
```

- The `csv_data` folder contains some .csv files which I downloaded from seneca
- The `mongo_db` folder is where the mongo_db should be stored in the end locally, so it will not be deleted after we stop the container (aka volume for this docker container)

In order to create and run a Docker container with MongoDB I changed my working directory into the `docker_test` folder and ran the following command in the terminal:
```
docker run -d -p 27017:27017 -v `pwd`/data/mongo_db:/data/db --name mongo_db mongo   
```
Now the container should be running in the background (you can check with ```docker container ls```)

## Python code to connect to MongoDB and insert data

In [1]:
# imports
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Client connects to "localhost" by default 
client = MongoClient()

In [3]:
# Create new client
db = client['TravelDashboard']

### Test whether we can add documents which will be persisted even after stopping and restarting the container

In [10]:
# this is just a random example of a docuemnt which could be entered
courses = {'title': 'Data Science',
         'lecturer': {
         'name': 'Markus Löcher',
         'department': 'Math',
         'status': 'Professor'}}

In [12]:
db.courses.insert_one(courses)

In between these steps I stopped an restarted the docker container:
```
docker stop mongo_db
docker restart mongo_db
```
Now let's check if the the entry is still there

In [13]:
# Print all documents
cursor = db.courses.find()

for document in cursor:
    pprint(document)

{'_id': ObjectId('62a8684d3138824bd05e4e39'),
 'lecturer': {'department': 'Math',
              'name': 'Markus Löcher',
              'status': 'Professor'},
 'title': 'Data Science'}


The entry is still there :) That's good.

In [14]:
# lets drop the courses collection
db.courses.drop()

### Add all sample .csv files to the MongoDB using python

In [23]:
# Check current directory where notebook is located
import os 
import glob
os.path.abspath("") # in python it should not be (""), but (__file__) !!!

'/Users/philippheitmann/Desktop/docker_test'

In [26]:
# get path of all .csv files in csv_data folder
all_files = glob.glob(os.path.join(os.path.abspath(""), "data", "csv_data", "*.csv"))

In [28]:
import pandas as pd
df = pd.read_csv(all_files[0], index_col=0)

In [75]:
#Store csv as list of dictonaries (each row/flight will be one dict)
df_dict = df.to_dict(orient='records')

In [84]:
# input the lsit of dicts into MongoDB
db.travel_data.insert_many(df_dict)

In between these steps I stopped an restarted the docker container:
```
docker stop mongo_db
docker restart mongo_db
```
Now let's check if the the entries are still there

In [95]:
# Count number of documents in MongoDB collection travel_data
db.travel_data.count_documents({})

3327

In [98]:
# Check if number of documents is equal to number of rows from the dataframe
db.travel_data.count_documents({})==df.shape[0]

True

### Insert all csv files in the mongodb
Great :) Now lets include all documents in the csv_folder into the mongoDB collection

In [107]:
# first let's drop the collection 
db.travel_data.drop()

In [108]:
# Lets see how big the mongo folder is before including the documents
!du -hs data/mongo_db     

 35M	data/mongo_db


In [109]:
%%time
# Loop over all files in csv_data folder and insert them into the MongoDB
for file in all_files:
    db.travel_data.insert_many(pd.read_csv(file, index_col=0).to_dict(orient='records'))

CPU times: user 5.54 s, sys: 201 ms, total: 5.74 s
Wall time: 12.1 s


So for 10 .csv files it took me ~12 seconds. In our seneca folder there are more than 16k files already :P 

In [110]:
# If we want to include all csv files into mongodb we will need 4.5 hours (for the current number of files)
16_000/60/60

4.444444444444445

In [111]:
# Lets see how big the mongo folder is after including the documents
!du -hs data/mongo_db  

 56M	data/mongo_db


So at least for this sample it was ~ 2MB per .csv file 

In [112]:
# So if we include all .csv files (current status: 16k) we will need 32GB of disk space on our local machine 
2*16_000/1000

32.0